In [1]:
# !ln -s /kaggle/input/caltech-birds-2011-dataset/CUB_200_2011/images ./filelists/CUB/images
# !ln -s /kaggle/input/miniimagenet/miniImageNet/images ./filelists/miniImagenet/images
# !ln -s /kaggle/input/d/arjun2000ashok/tieredimagenet/tiered_imagenet/ ./filelists/tieredImagenet/images
!ln -s /kaggle/input/stanford-dogs-dataset/images/Images ./filelists/dogs/Images
# !ln -s /kaggle/input/stanford-cars-dataset/cars_train/cars_train ./filelists/cars/images 
# !ln -s /kaggle/input/fgvc-aircraft/fgvc-aircraft-2013b/fgvc-aircraft-2013b/data/images ./filelists/aircrafts/images
# !ln -s /kaggle/input/vggflowers/images ./filelists/flowers/images

In [3]:
!python train.py --dataset dogs --train_aug --method baseline --tracking --stop_epoch 400
!python train.py --dataset dogs --train_aug --method baseline --tracking --stop_epoch 400 --resume --resume_wandb_id 27wluxlz

tracking in baseline train: True
USE pre-trained model: False
Checkpoint path: ckpts/dogs/_resnet18_baseline_aug_tracking_lr0.0010
Fresh wandb run
wandb: Currently logged in as: multi-input (use `wandb login --relogin` to force relogin)
^C
Problem at: train.py 255 <module>
Fatal Python error: initsite: Failed to import the site module
Traceback (most recent call last):
  File "/opt/conda/lib/python3.7/site.py", line 579, in <module>
Fatal Python error: initsite: Failed to import the site module
Traceback (most recent call last):
  File "/opt/conda/lib/python3.7/site.py", line 579, in <module>
    main()
  File "/opt/conda/lib/python3.7/site.py", line 566, in main
    known_paths = addsitepackages(known_paths)
  File "/opt/conda/lib/python3.7/site.py", line 349, in addsitepackages
    addsitedir(sitedir, known_paths)
  File "/opt/conda/lib/python3.7/site.py", line 207, in addsitedir
    addpackage(sitedir, name, known_paths)
  File "/opt/conda/lib/python3.7/site.py", line 168, in addpac

In [7]:
## To rename the saved epoch file

import torch

a = torch.load("ckpts/dogs/_resnet18_baseline_aug_tracking_lr0.0010/last_model.tar")
print(a['epoch'])

349


In [ ]:
!python save_features.py --dataset dogs --train_aug --method baseline --tracking --save_iter 399

In [ ]:
!python test.py --dataset dogs --train_aug --method baseline --tracking --save_iter 399